In [1]:
import math
import warnings

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.model_selection import train_test_split

%matplotlib inline
sns.set_style('white')

# Display preferences.
pd.options.display.float_format = '{:.3f}'.format


# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [2]:
nycrime = pd.read_csv('./NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013 - 13tbl8ny.csv',thousands=',')
nycrime_valid = pd.read_csv('./Table_8_Offenses_Known_to_Law_Enforcement_by_New_York_by_City_2014.csv',thousands=',')

In [3]:
nycrime_df = nycrime[['Propertycrime','Population','Murder','Robbery']]
nycrime_valid_df = nycrime_valid[['Propertycrime','Population','Murder','Robbery']]

In [4]:
nycrime_df = nycrime_df.astype(float)
nycrime_valid_df = nycrime_valid_df.astype(float)

In [5]:
nycrime_valid_df.dtypes

Propertycrime    float64
Population       float64
Murder           float64
Robbery          float64
dtype: object

In [6]:
nycrime_df.head()

,Propertycrime,Population,Murder,Robbery
0,12.000,1861.000,0.000,0.000
1,24.000,2577.000,0.000,0.000
2,16.000,2846.000,0.000,0.000
3,4090.000,97956.000,8.000,227.000
4,223.000,6388.000,0.000,4.000


In [7]:
nycrime_valid_df.head()

,Propertycrime,Population,Murder,Robbery
0,11.000,1851.000,0.000,0.000
1,49.000,2568.000,0.000,1.000
2,1.000,820.000,0.000,0.000
3,17.000,2842.000,0.000,0.000
4,3888.000,98595.000,8.000,237.000


In [8]:
nycrime_df['City'] = nycrime['City']
nycrime_valid_df['City'] = nycrime_valid['City']

In [9]:
nycrime_df['popsq'] = nycrime_df['Population']**2
nycrime_df['Robberies'] = np.where(nycrime_df['Robbery']>0,1,0)
nycrime_df['Murders'] = np.where(nycrime_df['Murder']>0,1,0)
nycrime_df['bigMurder'] = np.where(nycrime_df['Murder']>20,1,0)
nycrime_df['bigRob'] = np.where(nycrime_df['Robbery'] > 400,1,0)
nycrime_df['midRob'] = np.where((nycrime_df['Robbery'] < 400) & (nycrime_df['Robbery'] > 100),1,0)
nycrime_df['lowPop'] = np.where((nycrime_df['Population'] < 150000),1,0)
nycrime_df['midMurd'] = np.where((nycrime_df['Murder'] < 15) & (nycrime_df['Murder'] > 5),1,0)
nycrime_df['lowPopxmurder'] =nycrime_df["lowPop"]*nycrime_df['Murder']
nycrime_df['lowPopxrob'] =nycrime_df["lowPop"]*nycrime_df['Murder']

######################################################################3
nycrime_valid_df['popsq'] = nycrime_valid_df['Population']**2
nycrime_valid_df['Robberies'] = np.where(nycrime_valid_df['Robbery']>0,1,0)
nycrime_valid_df['Murders'] = np.where(nycrime_valid_df['Murder']>0,1,0)
nycrime_valid_df['bigMurder'] = np.where(nycrime_valid_df['Murder']>20,1,0)
nycrime_valid_df['bigRob'] = np.where(nycrime_valid_df['Robbery'] > 400,1,0)
nycrime_valid_df['midRob'] = np.where((nycrime_valid_df['Robbery'] < 400) & (nycrime_valid_df['Robbery'] > 100),1,0)
nycrime_valid_df['lowPop'] = np.where((nycrime_valid_df['Population'] < 150000),1,0)
nycrime_valid_df['midMurd'] = np.where((nycrime_valid_df['Murder'] < 15) & (nycrime_valid_df['Murder'] > 5),1,0)
nycrime_valid_df['lowPopxmurder'] =nycrime_valid_df["lowPop"]*nycrime_valid_df['Murder']
nycrime_valid_df['lowPopxrob'] =nycrime_valid_df["lowPop"]*nycrime_valid_df['Murder']

In [10]:
nycrime_df = nycrime_df[nycrime_df.City != 'New York']

In [11]:
nycrime_valid_df.iloc[227,:]

Propertycrime           135747.000
Population             8473938.000
Murder                     333.000
Robbery                  16581.000
City                     New York4
popsq           71807625227844.000
Robberies                        1
Murders                          1
bigMurder                        1
bigRob                           1
midRob                           0
lowPop                           0
midMurd                          0
lowPopxmurder                0.000
lowPopxrob                   0.000
Name: 227, dtype: object

In [12]:
nycrime_valid_df = nycrime_valid_df[nycrime_valid_df.City != 'New York4']

In [13]:
formula1 = 'Propertycrime ~ Population + Murder + Robbery + popsq + bigMurder + bigRob + lowPop + midMurd + midRob + Robberies + Murders + lowPopxmurder + lowPopxrob'

# Fit the model to our data using the formula.
lm = smf.ols(formula=formula1, data=nycrime_df).fit()

In [14]:
lm.params

Intercept       -4965.737
Population          0.015
Murder           -119.204
Robbery             3.656
popsq               0.000
bigMurder        5757.446
bigRob           5852.982
lowPop           4945.979
midMurd          1104.391
midRob            805.488
Robberies          83.915
Murders           227.982
lowPopxmurder     -33.660
lowPopxrob        -33.660
dtype: float64

In [15]:
lm.pvalues

Intercept       0.000
Population      0.000
Murder          0.355
Robbery         0.001
popsq           0.072
bigMurder       0.000
bigRob          0.201
lowPop          0.000
midMurd         0.000
midRob          0.000
Robberies       0.002
Murders         0.000
lowPopxmurder   0.614
lowPopxrob      0.614
dtype: float64

In [16]:
lm.rsquared

0.9617447843964877

* From the pvalues of the t-tests, we have weak evidence against the null of a zero coefficient for Murder, bigRob, lowPopxmurder, popsq, and lowPopxrob.

* This should simplify the regression considerably

In [17]:
formula2 = 'Propertycrime ~ Population + Robbery + bigMurder + lowPop + midMurd + midRob + Robberies + Murders' 

# Fit the model to our data using the formula.
lm2 = smf.ols(formula=formula2, data=nycrime_df).fit()

In [18]:
print(lm2.pvalues)
print(lm2.rsquared)

Intercept    0.000
Population   0.000
Robbery      0.000
bigMurder    0.000
lowPop       0.000
midMurd      0.235
midRob       0.594
Robberies    0.014
Murders      0.944
dtype: float64
0.9520022758451617


* midMurd, midRob, and Murders are insignificant.

* The difference to R^2 is minimal.

In [19]:
formula3 = 'Propertycrime ~ Population + Robbery + bigMurder + lowPop + Robberies' 

# Fit the model to our data using the formula.
lm3 = smf.ols(formula=formula3, data=nycrime_df).fit()

In [20]:
print(lm3.pvalues)
print(lm3.rsquared)

Intercept    0.000
Population   0.000
Robbery      0.000
bigMurder    0.000
lowPop       0.000
Robberies    0.011
dtype: float64
0.9518015699751828


* Very little change in R^2.

* Variables that are left are all significant.

* Smaller model will probably generalize better to new data.

In [21]:
from sklearn import linear_model
from sklearn import metrics

In [22]:
regr1 = linear_model.LinearRegression()
Y1 = nycrime_df['Propertycrime'].values.reshape(-1, 1)
X1 = nycrime_df[['Population','Robbery','bigMurder','lowPop','Robberies']]
regr1.fit(X1, Y1)

# Inspect the results.
print('\nCoefficients: \n', regr1.coef_)
print('\nIntercept: \n', regr1.intercept_)
print('\nR-squared:')
print(regr1.score(X1, Y1))


Coefficients: 
 [[1.51727729e-02 7.13106735e+00 2.09575272e+03 2.77699095e+03
  7.12530373e+01]]

Intercept: 
 [-2797.58092978]

R-squared:
0.9518015699751828


In [23]:
Y_valid1 = nycrime_valid_df['Propertycrime'].values.reshape(-1, 1)
X_valid1 = nycrime_valid_df[['Population','Robbery','bigMurder','lowPop','Robberies']]


In [24]:
regr1.score(X_valid1,Y_valid1)

# Inspect the results.

print('\nR-squared:')
print(regr1.score(X_valid1, Y_valid1))


R-squared:
0.939108765149033


* Very strong evidence this is a good model for New York State property crime prediction.

* The model fit on 2013 data has almost the same R squared on 2014 data.

* Just to be sure lets go the other way around.

* Fit model on 2014 data and evaluate on 2013.

In [25]:
regr2 = linear_model.LinearRegression()
Y1 = nycrime_df['Propertycrime'].values.reshape(-1, 1)
X1 = nycrime_df[['Population','Robbery','bigMurder','lowPop','Robberies']]
regr2.fit(X_valid1, Y_valid1)



# Inspect the results.
# Inspect the results.
print('\nCoefficients: \n', regr2.coef_)
print('\nIntercept: \n', regr2.intercept_)
print('\nR-squared:')
regr2.score(X1,Y1)


Coefficients: 
 [[1.74411343e-02 6.01456619e+00 4.26853533e+03 3.63882820e+03
  2.87061616e+01]]

Intercept: 
 [-3672.03098254]

R-squared:


0.9358168551285451

* Excellent results! Almost the same as before.

* We have shown that our smaller model without insignificant coefficients is robust and still fits well.

* The model fit on one year can fit out-of-sample on another years data almost as well as in sample results.

In [ ]:
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)

In [ ]:
cv = 10 # number of folds
n = int(len(data) / cv)
#print(len(data))
#print(n)
# Now loop through the chunks
results = []
for i in range(0,int(len(data)/n)):
    #print(i)
    ind = list(range(i,i+n))
    #print(ind)
    data_testfold = data.iloc[ind]
    data_trainfolds = data.iloc[~data.index.isin(ind)]
    target_testfold = target.iloc[ind]
    target_trainfolds = target.iloc[~target.index.isin(ind)]
    bnbcv = BernoulliNB()
    y_pred_cv = bnbcv.fit(data_trainfolds, target_trainfolds).predict(data_testfold)
    num_data = data_testfold.shape[0]
    correct = (num_data - (target_testfold != y_pred_cv).sum())/num_data * 100
    results.append(correct)
print(results)